In [1]:
!git clone https://github.com/RInkalshah93/ERA-V2-Assignment_Rinkal-Shah.git

Cloning into 'ERA-V2-Assignment_Rinkal-Shah'...
remote: Enumerating objects: 335, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 335 (delta 11), reused 25 (delta 10), pack-reused 309
Receiving objects: 100% (335/335), 35.38 MiB | 10.02 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [2]:
%cd ERA-V2-Assignment_Rinkal-Shah/S18_Assignment

/content/ERA-V2-Assignment_Rinkal-Shah/S16_Assignment


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_6

In [4]:
from config_file import get_config, get_weights_file_path
from train import get_model, get_ds, run_validation

config = get_config()
config["batch_size"] = 16
config["preload"] = None
config["num_epochs"] = 1
import torch
torch.cuda.amp.autocast(enabled = True)


/usr/local/lib/python3.10/dist-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [ ]:
import warnings
from tqdm import tqdm
import os
from pathlib import Path

from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)

Path(config["model_folder"]).mkdir(parent = True, exist_ok = True)

train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

writer = SummaryWriter(config['experiment_name'])
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAd]'), lable_smoothing=0.1).to(device)

optimizer = torch.optim.Adam(model.parameters(),lr = config['lr'], eps=1e-9)


In [ ]:
MAX_LR = 10**-3
STEPS_PER_EPOCH = len(train_dataloader)
EPOCHS = 18

In [5]:
train_model(config)

Using device : cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Max length of the source sentence : 309
Max length of the source target : 274
0


Processing Epoch 00: 100%|██████████| 1819/1819 [23:21<00:00,  1.30it/s, loss=6.497]


...

Processing Epoch 12: 100%|██████████| 1819/1819 [23:20<00:00,  1.30it/s, loss=2.854]


13


Processing Epoch 13: 100%|██████████| 1819/1819 [23:20<00:00,  1.30it/s, loss=2.872]


14


Processing Epoch 14: 100%|██████████| 1819/1819 [23:21<00:00,  1.30it/s, loss=2.723]
